In [1]:
import numpy as np
from torch.utils.data import DataLoader
from src.model.vae.conv_vae import ConvVAE
import torch
from src.utils.calculate_loss import calculate_test_loss, calculate_loss
from src.utils.load_data import load_data, normalize_data_tensor, recover_data_tensor
device = torch.device('cuda')
vae = ConvVAE(4).to(device)
vae.load_state_dict(torch.load('../../saved_models/SF/VAE-SF.torch'))

train,_,test = load_data('SF')
# mean = torch.tensor(train.mean(axis=(0,2,3))).cuda().float()
max = torch.tensor(np.array([train[:,i].max() for i in range(4)])).cuda().float()

mean = train.mean(axis=(0,2,3))
print(mean)


normalizer=max
device = torch.device('cuda')

dataloader_train = DataLoader(train, 32, shuffle=True, drop_last=True)
dataloader_test = DataLoader(test, 32, shuffle=False, drop_last=True)


[6.28927186 4.21874528 3.20243134 2.65271101]


In [2]:
train.mean()


4.090789871504157

calculate train test metrics

In [3]:
metric = ['RMSE', 'MSE', 'MAE']

In [4]:
vae.training=False
vae.eval()

metric = ['RMSE', 'MSE', 'MAE']
loss = {m:[] for m in metric}
for idx, data in enumerate(dataloader_train):
    data = data.cuda().float()
    data_recon,_,_ = vae(normalize_data_tensor(data,normalizer))
    for m in metric:
        l = torch.mean(calculate_loss(recover_data_tensor(data_recon, normalizer),
                                      data, m,
                                      reduction='none'),
                       dim=(0,2,3)).cpu().detach().numpy()
        loss[m].append(l)

for m in metric:
    loss[m] = np.stack(loss[m], axis=0)


In [5]:
loss_means = {m:loss[m].mean(axis=0) for m in metric}
loss_means

{'RMSE': array([1.8599832, 1.5543193, 1.1423467, 1.0126625], dtype=float32),
 'MSE': array([8.187742 , 5.9518037, 2.7818696, 2.2688549], dtype=float32),
 'MAE': array([1.8599832, 1.5543193, 1.1423467, 1.0126625], dtype=float32)}

In [6]:
loss_vars = {m:loss[m].var(axis=0) for m in metric}
loss_vars


{'RMSE': array([0.00378612, 0.0009655 , 0.0009055 , 0.00039442], dtype=float32),
 'MSE': array([0.41952336, 0.16268511, 0.03054917, 0.0094356 ], dtype=float32),
 'MAE': array([0.00378612, 0.0009655 , 0.0009055 , 0.00039442], dtype=float32)}

In [7]:
loss_test = {m:[] for m in metric}
for idx, data in enumerate(dataloader_test):
    data = data.cuda().float()
    data_recon,_,_ = vae(normalize_data_tensor(data,normalizer))
    for m in metric:
        l = torch.mean(calculate_loss(recover_data_tensor(data_recon, normalizer),
                                      data, m,
                                      reduction='none'),
                       dim=(0,2,3)).cpu().detach().numpy()
        loss_test[m].append(l)

for m in metric:
    loss_test[m] = np.stack(loss_test[m], axis=0)


In [8]:
loss_test_means = {m:loss_test[m].mean(axis=0) for m in metric}
loss_test_means

{'RMSE': array([1.8789412, 1.5952418, 1.1731682, 1.021729 ], dtype=float32),
 'MSE': array([8.368368 , 6.5952997, 2.9825232, 2.3105721], dtype=float32),
 'MAE': array([1.8789412, 1.5952418, 1.1731682, 1.021729 ], dtype=float32)}

In [9]:
loss_test_vars = {m:loss_test[m].var(axis=0) for m in metric}
loss_test_vars

{'RMSE': array([0.04178637, 0.02556471, 0.01404318, 0.0043015 ], dtype=float32),
 'MSE': array([3.7804139 , 4.217068  , 0.44750258, 0.12991385], dtype=float32),
 'MAE': array([0.04178637, 0.02556471, 0.01404318, 0.0043015 ], dtype=float32)}